In [ ]:
### Raphael Mourad
### Associate Professor
### University Paul Sabatier / INRAE MIAT Lab Toulouse
### 20/12/2024

In [1]:
# IMPORT LIBRARIES
import torch
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoConfig, AutoModelForPreTraining
from transformers import AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling, TextDataset
from datasets import load_dataset

from peft import PeftModel, PeftConfig, LoftQConfig
from peft import LoraConfig, get_peft_model, get_peft_model_state_dict

2024-01-16 17:39:45.446255: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 17:39:45.585961: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 17:39:46.002747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/lib/x86_64-linux-gnu
2024-01-16 17:39:46.002961: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.1: cannot o

In [2]:
torch.backends.cudnn.benchmark=True
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32 "

In [3]:
# SET DIRECTORY
os.chdir("/media/mourad/SSD2/MistralDNA/")
print(os.getcwd())

/media/mourad/SSD2/MistralDNA


In [4]:
#config = AutoConfig.from_pretrained("data/models/Mistral-7B-v0.1")
config = AutoConfig.from_pretrained("data/models/Mixtral-8x7B-v0.1")
model = AutoModelForCausalLM.from_config(config,attn_implementation="flash_attention_2")
model

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes. No dtype was provided, you should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes. No dtype was provided, you should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator.


MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(4096, 256)
    (layers): ModuleList(
      (0): MixtralDecoderLayer(
        (self_attn): MixtralFlashAttention2(
          (q_proj): Linear(in_features=256, out_features=256, bias=False)
          (k_proj): Linear(in_features=256, out_features=256, bias=False)
          (v_proj): Linear(in_features=256, out_features=256, bias=False)
          (o_proj): Linear(in_features=256, out_features=256, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=256, out_features=64, bias=False)
          (experts): ModuleList(
            (0): MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=256, out_features=256, bias=False)
              (w2): Linear(in_features=256, out_features=256, bias=False)
              (w3): Linear(in_features=256, out_features=256, bias=False)
              (act_fn): SiLU()
    

In [5]:
# LOAD DNA LETTER TOKENIZER
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="songlab/gpn-arabidopsis")
print(tokenizer)

encoding = tokenizer("ATTGTGGGTCCCCC", padding="longest", truncation=True, return_tensors="pt")
print(encoding)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


PreTrainedTokenizerFast(name_or_path='songlab/gpn-arabidopsis', vocab_size=6, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'pad_token': '[PAD]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
{'input_ids': tensor([[2, 5, 5, 4, 5, 4, 4, 4, 5, 3, 3, 3, 3, 3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [6]:
# LOAD BPE LETTER TOKENIZER
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
tokenizer.padding_side  = 'left'
print(tokenizer)

encoding = tokenizer("ATTGTGGGTCCCCC", padding="longest", truncation=True, return_tensors="pt")
print(encoding)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


PreTrainedTokenizerFast(name_or_path='zhihan1996/DNABERT-2-117M', vocab_size=4096, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
{'input_ids': tensor([[   1, 2061,  281,  485,    6,    2]]), 'token_type_ids': tensor([[0, 0,

In [8]:
# NUMBER OF MODEL PARAMETERS
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Model size: {pytorch_total_params/1000**2:.1f}M parameters")

Model size: 105.0M parameters


In [9]:
# LOAD DATA 
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

dataset_text = load_dataset("csv", data_files="data/genome_sequences/hg38/sequences_hg38_200b.csv")
#dataset_text = load_dataset("csv", data_files="data/genome_sequences/hg38/sequences_hg38_200b_small.csv.gz")
#dataset_text = load_dataset("csv", data_files="data/genome_sequences/hg38/sequences_hg38_200b_verysmall.csv.gz")

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="longest", truncation=True, return_tensors="pt")

dataset = dataset_text.map(tokenize_function, batched=True)
print(dataset["train"])

train_size = int(0.8 * len(dataset["train"]))
test_size = len(dataset["train"]) - train_size
train_set, val_set = torch.utils.data.random_split(dataset["train"], [train_size, test_size])
train_set[1]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/14555310 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 14555310
})


{'text': 'TTTTATCGTGGTAAAATATATGTGACGGGAAATACACGACCTTAACCATTCTCGAGTGTGCGGCTCAGTGGCATTAATTCACAGTGCTTGAGGCCATCGCTGCTACCCATCTCCAGAACCCTTTCAACCTGCAAAACCCCACACCCACTCAACACTAACTCCCCACTCCCCTCTCCCAGGCCCTGGCAACCCCCACTCCA',
 'input_ids': [3,
  3,
  3,
  3,
  3,
  3,
  3,
  1,
  69,
  539,
  258,
  417,
  2275,
  45,
  80,
  93,
  516,
  173,
  40,
  2548,
  72,
  63,
  674,
  453,
  44,
  924,
  4092,
  1330,
  37,
  95,
  75,
  151,
  232,
  56,
  74,
  96,
  204,
  1162,
  164,
  1071,
  2325,
  78,
  1745,
  53,
  205,
  66,
  906,
  326,
  2],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [10]:
len(train_set)

11644248

In [11]:
# PARAMETERS FOR FINE-TUNING
batchsize=1024
training_args = TrainingArguments(
        output_dir='./results/models',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=50,
        per_device_train_batch_size=batchsize,
        per_device_eval_batch_size=batchsize,
        learning_rate=5e-4,
        weight_decay=0.01,
        logging_dir='./logs',
        load_best_model_at_end=True,
        fp16=True,
        gradient_accumulation_steps=50,
)

print(training_args)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=50,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=False,
group_by

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# PRETRAIN MODEL
# with 105Mb Mixtral model, 110h / 50 epochs (1M sequences) = 2h / epoch
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_set,
    eval_dataset=val_set,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)
 
print ('Start a trainer...')
# Start training
trainer.train()

Start a trainer...


Epoch,Training Loss,Validation Loss
0,No log,6.240944
1,No log,5.840853
2,6.267100,5.713866
3,6.267100,5.635575
4,5.700700,5.646897
5,5.700700,5.588204
6,5.577000,5.526154
7,5.577000,5.503280
8,5.521000,5.491899
9,5.521000,5.471615


In [ ]:
input_string=["CGACCCTACTTCCCGCGGCCCCGGACGCCTCCTCACCTGCGAGCCGCCCTCCCGGAAGCTCCCGCCGCCGCTTCCGCT[MASK]TGCCGGAGCCGCTGGGTCCTAGCCCCGCCGCCCACAGTCCGCCCGCGCCTCCGGGTCCTAACGCCGCCGCTCGCCCTCCGCTGCGCCCTCCCCGAGCGCGGCTCCAGGACCCCGTCGACCC"]*2
input_string

tokenized_inputs = tokenizer(input_string, return_tensors="pt")
outputs=model(tokenized_inputs["input_ids"].cuda())
outputs

probs=torch.nn.functional.softmax(outputs['logits'],2)
probs

In [ ]:
input_string=["CGACCCTACTTCCCGCGGCC[MASK]TGCCGTCCAGGACCCCGTCGACCC"]*2
input_string

tokenized_inputs = tokenizer(input_string, return_tensors="pt")
outputs=model(tokenized_inputs["input_ids"].cuda())
outputs

probs=torch.nn.functional.softmax(outputs['logits'],2)
probs